In [2]:
import os
import tarfile
import urllib.request

DOWNLOAD_ROOT = "http://spamassassin.apache.org/old/publiccorpus/"
# HAM_URL = DOWNLOAD_ROOT + "20030228_easy_ham.tar.bz2"
# SPAM_URL = DOWNLOAD_ROOT + "20030228_spam.tar.bz2"
SPAM_PATH = os.path.join("DataSets", "spam")
HAM_PATH = os.path.join("DataSets", "ham")

HAM_LIST = ['20021010_hard_ham.tar.bz2', '20030228_hard_ham.tar.bz2'] 
SPAM_LIST = ['20021010_spam.tar.bz2', '20050311_spam_2.tar.bz2']
 

def fetch_spam_data(hams, spams, ham_path, spam_path):
    if not os.path.isdir(spam_path):
        os.makedirs(spam_path)
        os.makedirs(ham_path)
        
    for file in hams:
        url = DOWNLOAD_ROOT + file
        path = os.path.join(ham_path, file)
        if not os.path.isfile(path):
            urllib.request.urlretrieve(url, path)
        tar_file = tarfile.open(path)
        tar_file.extractall(path=ham_path)
        tar_file.close()
        
    for file in spams:
        url = DOWNLOAD_ROOT + file
        path = os.path.join(spam_path, file)
        if not os.path.isfile(path):
            urllib.request.urlretrieve(url, path)
        tar_file = tarfile.open(path)
        tar_file.extractall(path=spam_path)
        tar_file.close()
        
        
fetch_spam_data(HAM_LIST, SPAM_LIST, HAM_PATH, SPAM_PATH)

In [3]:
import email
import email.policy


HAM_SRC = 'DataSets/ham/Ham/'
SPAM_SRC = 'DataSets/spam/Spam/'

hams = os.listdir(HAM_SRC)
spams = os.listdir(SPAM_SRC)

ham_contents = []
spam_contents = []

for ham in hams:
    ham_email = os.path.join(HAM_SRC, ham)
    with open(ham_email, 'rb') as file:
        ham_content = email.parser.BytesParser(policy=email.policy.default).parse(file)
        ham_contents.append(ham_content)
        
for spam in spams:
    spam_email = os.path.join(SPAM_SRC, spam)
    with open(spam_email, 'rb') as file:
        spam_content = email.parser.BytesParser(policy=email.policy.default).parse(file)
        spam_contents.append(spam_content)
        
print(len(ham_contents))
print(len(spam_contents))

501
1898


In [4]:
test_email = ham_contents[0]
print(test_email.get_content().strip())

======================== THE MOTLEY FOOL ========================
                        PERSONAL  FINANCE 
                    Wednesday, January 2, 2002 
										 
mkettler@home.com

IN THIS ISSUE

- ASK THE FOOL: Stop the Solicitation!
  http://www.fool.com/m.asp?i=562064

- YOUR MONEY: Resolutions You Can Keep
  http://www.fool.com/m.asp?i=562065

- SPOTLIGHT: Should All My Savings Be in the Market?
  http://www.fool.com/m.asp?i=562066
	
- COMMUNITY TIPS: Do (At Least) One Thing Differently
  http://www.fool.com/m.asp?i=562067


SPONSORED BY: Datek
Datek Online. Built to trade.
*Proprietary auto routing technology | *$9.99 commission 
for online equity trades | *60-second execution commitment.
Apply for a Datek Account Now!
http://www.lnksrv.com/m.asp?i=562068

ASK THE FOOL

Q. I'M TIRED OF GETTING SO MANY CREDIT CARD OFFERS IN THE MAIL. 
I ALREADY HAVE 
A CARD AND I DON'T WANT ANY MORE MAIL OFFERING ME NEW ONES. HOW 
CAN I STOP IT? 

A. There are three main credit bureaus in th

In [5]:
print(test_email.get_payload())


======================== THE MOTLEY FOOL ========================
                        PERSONAL  FINANCE 
                    Wednesday, January 2, 2002 
										 
mkettler@home.com

IN THIS ISSUE

- ASK THE FOOL: Stop the Solicitation!
  http://www.fool.com/m.asp?i=562064

- YOUR MONEY: Resolutions You Can Keep
  http://www.fool.com/m.asp?i=562065

- SPOTLIGHT: Should All My Savings Be in the Market?
  http://www.fool.com/m.asp?i=562066
	
- COMMUNITY TIPS: Do (At Least) One Thing Differently
  http://www.fool.com/m.asp?i=562067


SPONSORED BY: Datek
Datek Online. Built to trade.
*Proprietary auto routing technology | *$9.99 commission 
for online equity trades | *60-second execution commitment.
Apply for a Datek Account Now!
http://www.lnksrv.com/m.asp?i=562068

ASK THE FOOL

Q. I'M TIRED OF GETTING SO MANY CREDIT CARD OFFERS IN THE MAIL. 
I ALREADY HAVE 
A CARD AND I DON'T WANT ANY MORE MAIL OFFERING ME NEW ONES. HOW 
CAN I STOP IT? 

A. There are three main credit bureaus in t

In [6]:
print(len(test_email.get_payload()))
print(len(test_email.get_content()))

7050
7050


In [7]:
from collections import Counter

def get_email_structure(email):
    if isinstance(email, str):
        return email
    payload = email.get_payload()
    if isinstance(payload, list):
        return "multipart({})".format(", ".join([
            get_email_structure(sub_email)
            for sub_email in payload
        ]))
    else:
        return email.get_content_type()


def structures_counter(emails):
    structures = Counter()
    for email in emails:
        structure = get_email_structure(email)
        structures[structure] += 1
    return structures

In [8]:
structures_counter(ham_contents).most_common()

[('text/html', 238),
 ('text/plain', 168),
 ('multipart(text/plain, text/html)', 81),
 ('multipart(text/html)', 4),
 ('multipart(text/plain, image/bmp)', 2),
 ('multipart(multipart(text/plain, text/html))', 2),
 ('multipart(text/plain, application/x-pkcs7-signature)', 2),
 ('multipart(text/plain, image/png, image/png)', 2),
 ('multipart(multipart(text/plain, text/html), image/gif, image/gif, image/gif, image/gif, image/gif, image/gif, image/gif, image/gif, image/gif, image/gif, image/gif, image/jpeg, image/gif, image/gif, image/gif, image/gif, image/gif, image/gif)',
  1),
 ('multipart(text/plain, text/plain)', 1)]

In [9]:
structures_counter(spam_contents).most_common()

[('text/plain', 820),
 ('text/html', 770),
 ('multipart(text/plain, text/html)', 159),
 ('multipart(text/html)', 48),
 ('multipart(text/plain)', 44),
 ('multipart(multipart(text/html))', 23),
 ('multipart(multipart(text/plain, text/html))', 5),
 ('multipart(text/plain, application/octet-stream, text/plain)', 3),
 ('multipart(text/plain, application/octet-stream)', 3),
 ('multipart(text/html, text/plain)', 3),
 ('multipart(text/plain, image/jpeg)', 3),
 ('multipart(text/html, image/jpeg)', 2),
 ('multipart(multipart(text/plain), application/octet-stream)', 2),
 ('multipart/alternative', 2),
 ('multipart(text/html, application/octet-stream)', 2),
 ('multipart(text/plain, multipart(text/plain))', 1),
 ('multipart(multipart(text/plain, text/html), image/jpeg, image/jpeg, image/jpeg, image/jpeg, image/jpeg)',
  1),
 ('multipart(multipart(text/plain, text/html), image/jpeg, image/jpeg, image/jpeg, image/jpeg, image/gif)',
  1),
 ('text/plain charset=us-ascii', 1),
 ('multipart(multipart(text

In [10]:
for key in test_email:
    print(key)

Return-Path
Delivery-Date
Return-Path
Received
Received
Received
Message-Id
Received
To
From
Reply-To
Date
Subject
Content-Type
X-Eudora2Unix


In [11]:
test_email['From']

'The Motley Fool <Fool@motleyfool.com>'

In [12]:
import re
from html import unescape

def html_to_plain_text(html):
    text = re.sub('<head.*?>.*?</head>', '', html, flags=re.M | re.S | re.I)
    text = re.sub('<a\s.*?>', ' HYPERLINK ', text, flags=re.M | re.S | re.I)
    text = re.sub('<.*?>', '', text, flags=re.M | re.S)
    text = re.sub(r'(\s*\n)+', '\n', text, flags=re.M | re.S)
    return unescape(text)

In [13]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split, StratifiedShuffleSplit

data = np.array(ham_contents + spam_contents, dtype=object)
target = np.array([0]*len(ham_contents) + [1]*len(spam_contents), dtype=np.int16)

# X_train, X_test, y_train, y_test = train_test_split(data, target, test_size=0.2, random_state=69)

split = StratifiedShuffleSplit(n_splits=1, test_size=0.2, random_state=69)

for train_idx, test_idx in split.split(data, target):
    train_data = data[train_idx]
    test_data = data[test_idx]
    train_target = target[train_idx]
    test_target = target[test_idx]
    

print(sum(test_target == 0)/len(test_target))
sum(target == 0)/len(target)

0.20833333333333334


0.20883701542309296

In [148]:
def email_to_text(email):
    s = ' '
    for part in email.walk():
        payload = part.get_payload()
        if isinstance(payload, str):
            s += payload + ' '
        elif isinstance(payload, list):
            for each in payload:
                s += email_to_text(each) or ' '
    return s

print(email_to_text(test_email))

 
======================== THE MOTLEY FOOL ========================
                        PERSONAL  FINANCE 
                    Wednesday, January 2, 2002 
										 
mkettler@home.com

IN THIS ISSUE

- ASK THE FOOL: Stop the Solicitation!
  http://www.fool.com/m.asp?i=562064

- YOUR MONEY: Resolutions You Can Keep
  http://www.fool.com/m.asp?i=562065

- SPOTLIGHT: Should All My Savings Be in the Market?
  http://www.fool.com/m.asp?i=562066
	
- COMMUNITY TIPS: Do (At Least) One Thing Differently
  http://www.fool.com/m.asp?i=562067


SPONSORED BY: Datek
Datek Online. Built to trade.
*Proprietary auto routing technology | *$9.99 commission 
for online equity trades | *60-second execution commitment.
Apply for a Datek Account Now!
http://www.lnksrv.com/m.asp?i=562068

ASK THE FOOL

Q. I'M TIRED OF GETTING SO MANY CREDIT CARD OFFERS IN THE MAIL. 
I ALREADY HAVE 
A CARD AND I DON'T WANT ANY MORE MAIL OFFERING ME NEW ONES. HOW 
CAN I STOP IT? 

A. There are three main credit bureaus in 

In [149]:
import nltk
from scipy.sparse import csr_matrix
from sklearn.base import BaseEstimator, TransformerMixin

In [150]:
import urlextract

url_extractor = urlextract.URLExtract()
stemmer = nltk.PorterStemmer()

def replace_urls(s):
    urls = url_extractor.find_urls(s)
    urls = list(set(urls))
    urls.sort(key=lambda x: len(x), reverse=True)
    for url in urls:
        s = s.replace(url, " URL ")
    
    return s

def count_words(s):
    s = s.lower()
    s = replace_urls(s)
    s = re.sub(r'\d+', ' NUM ', s)
    s = re.sub(r'\W+', ' ', s, flags=re.MULTILINE)
    word_list = s.split()
    stemmed_list = [stemmer.stem(word) for word in word_list]
    word_count = Counter(stemmed_list)
    return word_count

count_words(email_to_text(test_email)).most_common()

[('num', 48),
 ('the', 29),
 ('to', 29),
 ('url', 24),
 ('your', 21),
 ('a', 19),
 ('i', 17),
 ('in', 16),
 ('and', 16),
 ('for', 15),
 ('save', 13),
 ('of', 13),
 ('you', 12),
 ('that', 11),
 ('my', 10),
 ('one', 10),
 ('get', 10),
 ('money', 9),
 ('can', 9),
 ('t', 9),
 ('s', 9),
 ('at', 8),
 ('go', 8),
 ('have', 7),
 ('more', 7),
 ('it', 7),
 ('resolut', 6),
 ('keep', 6),
 ('be', 6),
 ('market', 6),
 ('do', 6),
 ('datek', 6),
 ('don', 6),
 ('are', 6),
 ('our', 6),
 ('we', 6),
 ('fool', 5),
 ('thi', 5),
 ('all', 5),
 ('so', 5),
 ('if', 5),
 ('list', 5),
 ('is', 5),
 ('term', 5),
 ('on', 5),
 ('or', 5),
 ('out', 5),
 ('financ', 4),
 ('ask', 4),
 ('tip', 4),
 ('least', 4),
 ('onlin', 4),
 ('trade', 4),
 ('now', 4),
 ('offer', 4),
 ('mail', 4),
 ('new', 4),
 ('they', 4),
 ('too', 4),
 ('short', 4),
 ('make', 4),
 ('person', 3),
 ('home', 3),
 ('should', 3),
 ('thing', 3),
 ('differ', 3),
 ('second', 3),
 ('account', 3),
 ('credit', 3),
 ('card', 3),
 ('want', 3),
 ('from', 3),
 ('phone'

In [151]:
class EmailToWordCounter(BaseEstimator, TransformerMixin):
    def __init__(self):
        pass
    
    def fit(self, X):
        return self
    
    def transform(self, X):
        X_transformed = []
        for instance in X:
            content = email_to_text(instance) or ''
            word_counter = count_words(content)
            X_transformed.append(word_counter)
        return np.array(X_transformed)             
            

In [152]:
class WordCounterToVector(BaseEstimator, TransformerMixin):
    def __init__(self, vocabulary_size):
        self.vocabulary_ = []
        self.vocabulary_size = vocabulary_size
    
    def fit(self, X):
        all_words = Counter()
        for counter in X:
            for word, count in counter.items():
                all_words[word] = +count
        self.vocabulary_ = [word for word, _ in all_words.most_common()[:self.vocabulary_size]]
        return self
    
    def transform(self, X):
        X_transformed = []
        for counter in X:
            x_transformed = [0] * (self.vocabulary_size + 1)
            for word, count in counter.items():
                if word in self.vocabulary_:
                    x_transformed[self.vocabulary_.index(word) + 1] += count
                else:
                    x_transformed[0] += count
            X_transformed.append(x_transformed)
        
        return csr_matrix(X_transformed)

In [153]:
from sklearn.pipeline import Pipeline

pipeline = Pipeline([
    ('to_counter', EmailToWordCounter()),
    ('to_vector', WordCounterToVector(10000))
])


X_transformed = pipeline.fit_transform(train_data)

print(X_transformed)

  (0, 0)	55
  (0, 3)	5
  (0, 4)	8
  (0, 6)	3
  (0, 13)	2
  (0, 14)	5
  (0, 19)	22
  (0, 27)	2
  (0, 28)	8
  (0, 43)	3
  (0, 45)	1
  (0, 48)	5
  (0, 64)	1
  (0, 84)	2
  (0, 93)	5
  (0, 112)	2
  (0, 121)	3
  (0, 126)	3
  (0, 147)	2
  (0, 209)	2
  (0, 210)	1
  (0, 259)	1
  (0, 260)	1
  (0, 300)	2
  (0, 301)	2
  :	:
  (1918, 8633)	4
  (1918, 8634)	4
  (1918, 9350)	2
  (1918, 9352)	2
  (1918, 9356)	2
  (1918, 9359)	2
  (1918, 9377)	2
  (1918, 9381)	2
  (1918, 9811)	2
  (1918, 9812)	2
  (1918, 9815)	2
  (1918, 9816)	2
  (1918, 9817)	2
  (1918, 9822)	2
  (1918, 9843)	2
  (1918, 9850)	2
  (1918, 9864)	2
  (1918, 9880)	2
  (1918, 9893)	2
  (1918, 9903)	2
  (1918, 9909)	2
  (1918, 9937)	2
  (1918, 9971)	2
  (1918, 9986)	2
  (1918, 9990)	2


In [154]:
X_transformed = X_transformed.toarray()
X_transformed

array([[ 55,   0,   0, ...,   0,   0,   0],
       [109,   0,   0, ...,   0,   0,   0],
       [ 21,   0,   0, ...,   0,   0,   0],
       ...,
       [ 11,   0,   0, ...,   0,   0,   0],
       [174,   0,   0, ...,   0,   0,   0],
       [ 74,   0,   0, ...,   0,   0,   0]], dtype=int32)

In [156]:
pipeline['to_vector'].vocabulary_[:200]

['mv',
 'iiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiii',
 'num',
 'a',
 'font',
 'the',
 'iiiiiiih',
 'url',
 'iiiiiiihepihepihf',
 'piiiiiiii',
 'td',
 'oracl',
 'p',
 'to',
 'tr',
 'kuxv',
 'size',
 'tabl',
 'br',
 's',
 'wdglvbj',
 'bgcolor',
 'color',
 'enenkio',
 'inset',
 'verdana',
 'href',
 'and',
 'width',
 'ffffff',
 'mailshot',
 'fxyq',
 'kingdom',
 'atol',
 'text',
 'g',
 'style',
 'face',
 'normal',
 'iiiiiiiiip',
 'arial',
 'iiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiij',
 'in',
 'border',
 'of',
 'marshal',
 'iih',
 'that',
 'cellspac',
 'cellpad',
 'vchrpb',
 'iiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiii',
 'iiiiiiiiiiiiiiiiiiiiiiip',
 'matrox',
 'homestor',
 'hover',
 'decor',
 'vivendi',
 'pgzvbnqgi',
 'infobytel',
 'it',
 'iiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiii',
 'mso',
 'on',
 'www',
 'wrox',
 'com',
 'famili',
 'small',
 'weight',
 'aol',
 'background',
 'votr',
 'kicagicagica',
 'winnick',
 'mm_swapimag',
 'mm_swapimg

In [160]:
np.sum(np.sum(X_transformed, axis=0) == 0)

0

In [167]:
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import cross_val_score

y_train = train_target

log_reg = LogisticRegression(max_iter=3000)
result = cross_val_score(log_reg, X_transformed, y_train, cv=3, scoring='accuracy')

result

array([0.9765625 , 0.9640625 , 0.97339593])

In [166]:
np.sum(y_train)/len(y_train)

0.7910369984366857

In [172]:
from sklearn.svm import SVC
from sklearn.model_selection import GridSearchCV


svc_param_grid = [
    {'kernel': ['rbf'], 'C': [0.5, 1.0, 5.0], 'gamma': ['scale', 'auto']},
    {'kernel': ['linear']  ,'C': [0.5, 1.0, 5.0]},
    {'kernel': ['poly'], 'C': [0.5, 1.0, 5.0], 'degree': [2, 3]}
]

svc_clf = SVC(max_iter=3000)
svc_grid_search = GridSearchCV(svc_clf, svc_param_grid, cv=10, scoring='accuracy', return_train_score=True)
svc_grid_search.fit(X_transformed, y_train)


c:\users\user\ds\venv\lib\site-packages\sklearn\svm\_base.py:255: ConvergenceWarning: Solver terminated early (max_iter=3000).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  warnings.warn('Solver terminated early (max_iter=%i).'
c:\users\user\ds\venv\lib\site-packages\sklearn\svm\_base.py:255: ConvergenceWarning: Solver terminated early (max_iter=3000).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  warnings.warn('Solver terminated early (max_iter=%i).'
c:\users\user\ds\venv\lib\site-packages\sklearn\svm\_base.py:255: ConvergenceWarning: Solver terminated early (max_iter=3000).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  warnings.warn('Solver terminated early (max_iter=%i).'
c:\users\user\ds\venv\lib\site-packages\sklearn\svm\_base.py:255: ConvergenceWarning: Solver terminated early (max_iter=3000).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  warnings.warn('Solver terminate

c:\users\user\ds\venv\lib\site-packages\sklearn\svm\_base.py:255: ConvergenceWarning: Solver terminated early (max_iter=3000).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  warnings.warn('Solver terminated early (max_iter=%i).'
c:\users\user\ds\venv\lib\site-packages\sklearn\svm\_base.py:255: ConvergenceWarning: Solver terminated early (max_iter=3000).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  warnings.warn('Solver terminated early (max_iter=%i).'
c:\users\user\ds\venv\lib\site-packages\sklearn\svm\_base.py:255: ConvergenceWarning: Solver terminated early (max_iter=3000).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  warnings.warn('Solver terminated early (max_iter=%i).'
c:\users\user\ds\venv\lib\site-packages\sklearn\svm\_base.py:255: ConvergenceWarning: Solver terminated early (max_iter=3000).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  warnings.warn('Solver terminate

GridSearchCV(cv=10, estimator=SVC(max_iter=3000),
             param_grid=[{'C': [0.5, 1.0, 5.0], 'gamma': ['scale', 'auto'],
                          'kernel': ['rbf']},
                         {'C': [0.5, 1.0, 5.0], 'kernel': ['linear']},
                         {'C': [0.5, 1.0, 5.0], 'degree': [2, 3],
                          'kernel': ['poly']}],
             return_train_score=True, scoring='accuracy')

In [174]:
svc_grid_search.best_params_

{'C': 0.5, 'kernel': 'linear'}

In [177]:
# cv_res = svc_grid_search.cv_results_

# print(cv_res.keys())

for score, params in zip(cv_res['mean_train_score'], cv_res['params']):
    print(score, '------', params)

0.9003530528212058 ------ {'C': 0.5, 'gamma': 'scale', 'kernel': 'rbf'}
0.95848519698042 ------ {'C': 0.5, 'gamma': 'auto', 'kernel': 'rbf'}
0.9058537203242617 ------ {'C': 1.0, 'gamma': 'scale', 'kernel': 'rbf'}
0.9738867912136225 ------ {'C': 1.0, 'gamma': 'auto', 'kernel': 'rbf'}
0.917144373673036 ------ {'C': 5.0, 'gamma': 'scale', 'kernel': 'rbf'}
0.9832089137124965 ------ {'C': 5.0, 'gamma': 'auto', 'kernel': 'rbf'}
0.9811260830170065 ------ {'C': 0.5, 'kernel': 'linear'}
0.9811260830170065 ------ {'C': 1.0, 'kernel': 'linear'}
0.9811260830170065 ------ {'C': 5.0, 'kernel': 'linear'}
0.8056859398121341 ------ {'C': 0.5, 'degree': 2, 'kernel': 'poly'}
0.8027908128525165 ------ {'C': 0.5, 'degree': 3, 'kernel': 'poly'}
0.816919024373244 ------ {'C': 1.0, 'degree': 2, 'kernel': 'poly'}
0.804875285498295 ------ {'C': 1.0, 'degree': 3, 'kernel': 'poly'}
0.8778300521134916 ------ {'C': 5.0, 'degree': 2, 'kernel': 'poly'}
0.8210876010637156 ------ {'C': 5.0, 'degree': 3, 'kernel': 'poly

In [178]:
log_clf = LogisticRegression(max_iter=4000)
log_clf.fit(X_transformed, y_train)

LogisticRegression(max_iter=4000)

In [181]:
X_test = pipeline.transform(test_data)

log_y_pred = log_clf.predict(X_test)
log_score = np.sum(log_y_pred == test_target)/len(test_target)
log_score

0.975

In [182]:
rbf_svm_clf = SVC(kernel='rbf', gamma='auto', C=5.0)
rbf_svm_clf.fit(X_transformed, y_train)

SVC(C=5.0, gamma='auto')

In [185]:
rbf_svm_y_pred = rbf_svm_clf.predict(X_test.toarray())
rbf_svm_score = np.sum(rbf_svm_y_pred == test_target)/len(test_target)
rbf_svm_score

0.9520833333333333

In [188]:
lin_svm_clf = SVC(kernel='linear', C=0.5)
lin_svm_clf.fit(X_transformed, y_train)

SVC(C=0.5, kernel='linear')

In [189]:
lin_svm_y_pred = rbf_svm_clf.predict(X_test.toarray())
lin_svm_score = np.sum(lin_svm_y_pred == test_target)/len(test_target)
lin_svm_score

0.9520833333333333